In [155]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr
from pyspark.sql import functions as f
from pyspark.sql.functions import substring
from pyspark.sql.functions import sum, sumDistinct
from pyspark.sql import SparkSession, DataFrame, Column, Row, GroupedData, \
    DataFrameNaFunctions, DataFrameStatFunctions, functions, types, Window
from pyspark.sql import functions as f
from pyspark.sql.functions import mean, stddev, variance, skewness, kurtosis

from pyspark.sql.functions import count, countDistinct, approx_count_distinct


## Some basic dataframe manipulation...

In [2]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()


In [3]:
df = spark.read.csv('./example_data.txt')

In [4]:
df.show(4)

+----+---+
| _c0|_c1|
+----+---+
|   n|  g|
|15.0|  b|
|23.0|  c|
| 6.0|  c|
+----+---+
only showing top 4 rows



In [5]:
df = df.withColumnRenamed("_c0", "number").withColumnRenamed("_c1", "group")


In [6]:
df.show(5)

+------+-----+
|number|group|
+------+-----+
|     n|    g|
|  15.0|    b|
|  23.0|    c|
|   6.0|    c|
|   NaN|    c|
+------+-----+
only showing top 5 rows



In [7]:
df.printSchema()


root
 |-- number: string (nullable = true)
 |-- group: string (nullable = true)



In [8]:
df.withColumn("number", df.number.cast("integer")).show()


+------+-----+
|number|group|
+------+-----+
|  null|    g|
|    15|    b|
|    23|    c|
|     6|    c|
|  null|    c|
|    26|    b|
|    12|    b|
|     8|    a|
|    18|    c|
|    14|    a|
|    20|    c|
|    22|    a|
|    21|    a|
|     1|    c|
|     0|    a|
|    17|    b|
|     2|    a|
|     7|    a|
|    16|    b|
|    24|    b|
+------+-----+
only showing top 20 rows



In [9]:
df.show(3)

+------+-----+
|number|group|
+------+-----+
|     n|    g|
|  15.0|    b|
|  23.0|    c|
+------+-----+
only showing top 3 rows



In [10]:
df.withColumn('n_is_even', df.number % 2 == 0).show()

+------+-----+---------+
|number|group|n_is_even|
+------+-----+---------+
|     n|    g|     null|
|  15.0|    b|    false|
|  23.0|    c|    false|
|   6.0|    c|     true|
|   NaN|    c|    false|
|  26.0|    b|     true|
|  12.0|    b|     true|
|   8.0|    a|     true|
|  18.0|    c|     true|
|  14.0|    a|     true|
|  20.0|    c|     true|
|  22.0|    a|     true|
|  21.0|    a|    false|
|   1.0|    c|    false|
|   0.0|    a|     true|
|  17.0|    b|    false|
|   2.0|    a|     true|
|   7.0|    a|    false|
|  16.0|    b|     true|
|  24.0|    b|     true|
+------+-----+---------+
only showing top 20 rows



In [11]:
df.show()

+------+-----+
|number|group|
+------+-----+
|     n|    g|
|  15.0|    b|
|  23.0|    c|
|   6.0|    c|
|   NaN|    c|
|  26.0|    b|
|  12.0|    b|
|   8.0|    a|
|  18.0|    c|
|  14.0|    a|
|  20.0|    c|
|  22.0|    a|
|  21.0|    a|
|   1.0|    c|
|   0.0|    a|
|  17.0|    b|
|   2.0|    a|
|   7.0|    a|
|  16.0|    b|
|  24.0|    b|
+------+-----+
only showing top 20 rows



In [12]:
df.selectExpr('*', 'number % 2 = 0 as number_is_even').show()

+------+-----+--------------+
|number|group|number_is_even|
+------+-----+--------------+
|     n|    g|          null|
|  15.0|    b|         false|
|  23.0|    c|         false|
|   6.0|    c|          true|
|   NaN|    c|         false|
|  26.0|    b|          true|
|  12.0|    b|          true|
|   8.0|    a|          true|
|  18.0|    c|          true|
|  14.0|    a|          true|
|  20.0|    c|          true|
|  22.0|    a|          true|
|  21.0|    a|         false|
|   1.0|    c|         false|
|   0.0|    a|          true|
|  17.0|    b|         false|
|   2.0|    a|          true|
|   7.0|    a|         false|
|  16.0|    b|          true|
|  24.0|    b|          true|
+------+-----+--------------+
only showing top 20 rows



In [13]:
df.drop('group').show()


+------+
|number|
+------+
|     n|
|  15.0|
|  23.0|
|   6.0|
|   NaN|
|  26.0|
|  12.0|
|   8.0|
|  18.0|
|  14.0|
|  20.0|
|  22.0|
|  21.0|
|   1.0|
|   0.0|
|  17.0|
|   2.0|
|   7.0|
|  16.0|
|  24.0|
+------+
only showing top 20 rows



In [14]:
df.limit(10).show()

+------+-----+
|number|group|
+------+-----+
|     n|    g|
|  15.0|    b|
|  23.0|    c|
|   6.0|    c|
|   NaN|    c|
|  26.0|    b|
|  12.0|    b|
|   8.0|    a|
|  18.0|    c|
|  14.0|    a|
+------+-----+



# Using 311 Data...


In [61]:
spark = SparkSession.builder.master("local").appName("read").\
    enableHiveSupport().\
    getOrCreate()

In [62]:
df_case = spark.read.csv('./sa311/case.csv', sep=",", header=True, inferSchema=True)

In [63]:
df_case.show()

+----------+----------------+----------------+------------+---------+-------------------+-----------+----------------+--------------------+------------------+-----------+---------+--------------------+----------------+
|   case_id|case_opened_date|case_closed_date|SLA_due_date|case_late|      num_days_late|case_closed|   dept_division|service_request_type|          SLA_days|case_status|source_id|     request_address|council_district|
+----------+----------------+----------------+------------+---------+-------------------+-----------+----------------+--------------------+------------------+-----------+---------+--------------------+----------------+
|1014127332|     1/1/18 0:42|    1/1/18 12:29|9/26/20 0:42|       NO| -998.5087616000001|        YES|Field Operations|        Stray Animal|             999.0|     Closed| svcCRMLS|2315  EL PASO ST,...|               5|
|1014127333|     1/1/18 0:46|     1/3/18 8:11| 1/5/18 8:30|       NO|-2.0126041669999997|        YES|     Storm Water|Remova

In [64]:
df_case.printSchema()

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- SLA_due_date: string (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: integer (nullable = true)



In [65]:
df_dept = spark.read.csv('./sa311/dept.csv', sep=",", header=True, inferSchema=True)

In [66]:
df_case = df_case.withColumn("case_opened_date", 
                   f.to_timestamp(f.col("case_opened_date"), 
                                  "M/d/yy H:mm")).\
        withColumn("case_closed_date", 
                   f.to_timestamp(f.col("case_closed_date"),
                                  "M/d/yy H:mm")).\
        withColumn("SLA_due_date", 
                   f.to_timestamp(f.col("SLA_due_date"),
                                  "M/d/yy H:mm"))

In [67]:
df_case.printSchema()

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: timestamp (nullable = true)
 |-- case_closed_date: timestamp (nullable = true)
 |-- SLA_due_date: timestamp (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: integer (nullable = true)



In [68]:
df_case.show(2)

+----------+-------------------+-------------------+-------------------+---------+-------------------+-----------+----------------+--------------------+-----------+-----------+---------+--------------------+----------------+
|   case_id|   case_opened_date|   case_closed_date|       SLA_due_date|case_late|      num_days_late|case_closed|   dept_division|service_request_type|   SLA_days|case_status|source_id|     request_address|council_district|
+----------+-------------------+-------------------+-------------------+---------+-------------------+-----------+----------------+--------------------+-----------+-----------+---------+--------------------+----------------+
|1014127332|2018-01-01 00:42:00|2018-01-01 12:29:00|2020-09-26 00:42:00|       NO| -998.5087616000001|        YES|Field Operations|        Stray Animal|      999.0|     Closed| svcCRMLS|2315  EL PASO ST,...|               5|
|1014127333|2018-01-01 00:46:00|2018-01-03 08:11:00|2018-01-05 08:30:00|       NO|-2.012604166999999

## How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

In [69]:
df_case.printSchema()

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: timestamp (nullable = true)
 |-- case_closed_date: timestamp (nullable = true)
 |-- SLA_due_date: timestamp (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: integer (nullable = true)



In [70]:
row = df_case.select('SLA_days', 'num_days_late').sort(df_case.SLA_days, df_case.num_days_late, ascending=False).first()
row

Row(SLA_days=1420.0, num_days_late=-1417.0006019999998)

In [71]:
the_most_SLA = row['SLA_days']
the_most_SLA

1420.0

In [72]:
the_most_numb_days_late = row['num_days_late']
the_most_numb_days_late

-1417.0006019999998

## How many Stray Animal cases are there?


In [73]:
df_case.printSchema()

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: timestamp (nullable = true)
 |-- case_closed_date: timestamp (nullable = true)
 |-- SLA_due_date: timestamp (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: integer (nullable = true)



In [74]:
df_case.describe()

DataFrame[summary: string, case_id: string, case_late: string, num_days_late: string, case_closed: string, dept_division: string, service_request_type: string, SLA_days: string, case_status: string, source_id: string, request_address: string, council_district: string]

In [75]:
df_case.where(df_case.service_request_type == 'Stray Animal').count()

26760

### How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

In [76]:
df_join = df_case.join(df_dept, "dept_division", "inner")

In [77]:
df_join.show()

+----------------+----------+-------------------+-------------------+-------------------+---------+-------------------+-----------+--------------------+------------------+-----------+---------+--------------------+----------------+--------------------+----------------------+-------------------+
|   dept_division|   case_id|   case_opened_date|   case_closed_date|       SLA_due_date|case_late|      num_days_late|case_closed|service_request_type|          SLA_days|case_status|source_id|     request_address|council_district|           dept_name|standardized_dept_name|dept_subject_to_SLA|
+----------------+----------+-------------------+-------------------+-------------------+---------+-------------------+-----------+--------------------+------------------+-----------+---------+--------------------+----------------+--------------------+----------------------+-------------------+
|Field Operations|1014127332|2018-01-01 00:42:00|2018-01-01 12:29:00|2020-09-26 00:42:00|       NO| -998.5087616

In [78]:
# pd_join  = df_join.toPandas().head()

In [79]:
# pd_join.head()

In [80]:
ans = df_join.where(df_join.dept_division == 'Field Operations').where(df_join.service_request_type != 'Officer Standby')

In [81]:
# ans.toPandas().head(5)

In [82]:
df_join.where(df_join.dept_division == 'Field Operations').where(df_join.service_request_type != 'Officer Standby').count()

113902

### Create a new DataFrame without any information related to dates or location.


In [83]:
df_join.dtypes

[('dept_division', 'string'),
 ('case_id', 'int'),
 ('case_opened_date', 'timestamp'),
 ('case_closed_date', 'timestamp'),
 ('SLA_due_date', 'timestamp'),
 ('case_late', 'string'),
 ('num_days_late', 'double'),
 ('case_closed', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'int'),
 ('dept_name', 'string'),
 ('standardized_dept_name', 'string'),
 ('dept_subject_to_SLA', 'string')]

In [84]:
drop = ['request_address']
for element in df_join.dtypes:
    if element[1] == 'timestamp':
        drop.append(element[0])
drop

['request_address', 'case_opened_date', 'case_closed_date', 'SLA_due_date']

In [85]:
# basically make a copy of the df_join, to make a new df to manipulate
df_nodates_location = df_join

In [86]:
for element in drop:
    df_nodates_location =  df_nodates_location.drop(element)
df_nodates_location.show()
df_nodates_location.printSchema()

+----------------+----------+---------+-------------------+-----------+--------------------+------------------+-----------+---------+----------------+--------------------+----------------------+-------------------+
|   dept_division|   case_id|case_late|      num_days_late|case_closed|service_request_type|          SLA_days|case_status|source_id|council_district|           dept_name|standardized_dept_name|dept_subject_to_SLA|
+----------------+----------+---------+-------------------+-----------+--------------------+------------------+-----------+---------+----------------+--------------------+----------------------+-------------------+
|Field Operations|1014127332|       NO| -998.5087616000001|        YES|        Stray Animal|             999.0|     Closed| svcCRMLS|               5|Animal Care Services|  Animal Care Services|                YES|
|     Storm Water|1014127333|       NO|-2.0126041669999997|        YES|Removal Of Obstru...|       4.322222222|     Closed| svcCRMSS|       

### Read dept.csv into a Spark DataFrame. Inspect the dept_name column. Replace the missing values with "other".


In [87]:
df_dept = spark.read.csv('./sa311/dept.csv', sep=",", header=True, inferSchema=True)
df_dept.show()

+--------------------+--------------------+----------------------+-------------------+
|       dept_division|           dept_name|standardized_dept_name|dept_subject_to_SLA|
+--------------------+--------------------+----------------------+-------------------+
|     311 Call Center|    Customer Service|      Customer Service|                YES|
|               Brush|Solid Waste Manag...|           Solid Waste|                YES|
|     Clean and Green|Parks and Recreation|    Parks & Recreation|                YES|
|Clean and Green N...|Parks and Recreation|    Parks & Recreation|                YES|
|    Code Enforcement|Code Enforcement ...|  DSD/Code Enforcement|                YES|
|Code Enforcement ...|Code Enforcement ...|  DSD/Code Enforcement|                YES|
|Code Enforcement ...|                null|  DSD/Code Enforcement|                YES|
|   Dangerous Premise|Code Enforcement ...|  DSD/Code Enforcement|                YES|
|Dangerous Premise...|Code Enforcement ...|

In [88]:
df_dept = df_dept.fillna('other', ['dept_name'])

In [89]:
df_dept.show()

+--------------------+--------------------+----------------------+-------------------+
|       dept_division|           dept_name|standardized_dept_name|dept_subject_to_SLA|
+--------------------+--------------------+----------------------+-------------------+
|     311 Call Center|    Customer Service|      Customer Service|                YES|
|               Brush|Solid Waste Manag...|           Solid Waste|                YES|
|     Clean and Green|Parks and Recreation|    Parks & Recreation|                YES|
|Clean and Green N...|Parks and Recreation|    Parks & Recreation|                YES|
|    Code Enforcement|Code Enforcement ...|  DSD/Code Enforcement|                YES|
|Code Enforcement ...|Code Enforcement ...|  DSD/Code Enforcement|                YES|
|Code Enforcement ...|               other|  DSD/Code Enforcement|                YES|
|   Dangerous Premise|Code Enforcement ...|  DSD/Code Enforcement|                YES|
|Dangerous Premise...|Code Enforcement ...|

### Convert the council_district column to a string column.


In [90]:
df_case.printSchema()

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: timestamp (nullable = true)
 |-- case_closed_date: timestamp (nullable = true)
 |-- SLA_due_date: timestamp (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: integer (nullable = true)



In [91]:
df_case.withColumn("council_district", df_case.council_district.cast("string")).printSchema()


root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: timestamp (nullable = true)
 |-- case_closed_date: timestamp (nullable = true)
 |-- SLA_due_date: timestamp (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: string (nullable = true)



In [92]:
df_join.printSchema()

root
 |-- dept_division: string (nullable = true)
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: timestamp (nullable = true)
 |-- case_closed_date: timestamp (nullable = true)
 |-- SLA_due_date: timestamp (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: integer (nullable = true)
 |-- dept_name: string (nullable = true)
 |-- standardized_dept_name: string (nullable = true)
 |-- dept_subject_to_SLA: string (nullable = true)



In [93]:
df_join.withColumn("council_district", df_join.council_district.cast("string")).printSchema()


root
 |-- dept_division: string (nullable = true)
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: timestamp (nullable = true)
 |-- case_closed_date: timestamp (nullable = true)
 |-- SLA_due_date: timestamp (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: string (nullable = true)
 |-- dept_name: string (nullable = true)
 |-- standardized_dept_name: string (nullable = true)
 |-- dept_subject_to_SLA: string (nullable = true)



### Extract the year from the case_closed_date column.

In [101]:
df_case.withColumn('year', substring('case_closed_date', 0, 4)).show()

+----------+-------------------+----------------+-------------------+---------+-------------------+-----------+----------------+--------------------+------------------+-----------+---------+--------------------+----------------+-------------------+----+
|   case_id|   case_opened_date|case_closed_date|       SLA_due_date|case_late|      num_days_late|case_closed|   dept_division|service_request_type|          SLA_days|case_status|source_id|     request_address|council_district|     num_hours_late|year|
+----------+-------------------+----------------+-------------------+---------+-------------------+-----------+----------------+--------------------+------------------+-----------+---------+--------------------+----------------+-------------------+----+
|1014127332|2018-01-01 00:42:00|            2018|2020-09-26 00:42:00|       NO| -998.5087616000001|        YES|Field Operations|        Stray Animal|             999.0|     Closed| svcCRMLS|2315  EL PASO ST,...|               5|     -2396

In [102]:
df_case = df_case.withColumn('year', substring('case_closed_date', 0, 4))

In [103]:
df_case.show()

+----------+-------------------+----------------+-------------------+---------+-------------------+-----------+----------------+--------------------+------------------+-----------+---------+--------------------+----------------+-------------------+----+
|   case_id|   case_opened_date|case_closed_date|       SLA_due_date|case_late|      num_days_late|case_closed|   dept_division|service_request_type|          SLA_days|case_status|source_id|     request_address|council_district|     num_hours_late|year|
+----------+-------------------+----------------+-------------------+---------+-------------------+-----------+----------------+--------------------+------------------+-----------+---------+--------------------+----------------+-------------------+----+
|1014127332|2018-01-01 00:42:00|            2018|2020-09-26 00:42:00|       NO| -998.5087616000001|        YES|Field Operations|        Stray Animal|             999.0|     Closed| svcCRMLS|2315  EL PASO ST,...|               5|     -2396

In [57]:
# df_case.withColumn('year', substring('case_closed_date', 0, 4).alias('year')).toPandas()

,case_id,case_opened_date,case_closed_date,SLA_due_date,case_late,num_days_late,case_closed,dept_division,service_request_type,SLA_days,case_status,source_id,request_address,council_district,year
0,1014127332,2018-01-01 00:42:00,2018-01-01 12:29:00,2020-09-26 00:42:00,NO,-998.508762,YES,Field Operations,Stray Animal,999.000000,Closed,svcCRMLS,"2315 EL PASO ST, San Antonio, 78207",5,2018
1,1014127333,2018-01-01 00:46:00,2018-01-03 08:11:00,2018-01-05 08:30:00,NO,-2.012604,YES,Storm Water,Removal Of Obstruction,4.322222,Closed,svcCRMSS,"2215 GOLIAD RD, San Antonio, 78223",3,2018
2,1014127334,2018-01-01 00:48:00,2018-01-02 07:57:00,2018-01-05 08:30:00,NO,-3.022338,YES,Storm Water,Removal Of Obstruction,4.320729,Closed,svcCRMSS,"102 PALFREY ST W, San Antonio, 78223",3,2018
3,1014127335,2018-01-01 01:29:00,2018-01-02 08:13:00,2018-01-17 08:30:00,NO,-15.011481,YES,Code Enforcement,Front Or Side Yard Parking,16.291887,Closed,svcCRMSS,"114 LA GARDE ST, San Antonio, 78223",3,2018
4,1014127336,2018-01-01 01:34:00,2018-01-01 13:29:00,2018-01-01 04:34:00,YES,0.372164,YES,Field Operations,Animal Cruelty(Critical),0.125000,Closed,svcCRMSS,"734 CLEARVIEW DR, San Antonio, 78228",7,2018
5,1014127337,2018-01-01 06:28:00,2018-01-01 14:38:00,2018-01-31 08:30:00,NO,-29.743981,YES,Signals,Traffic Signal Ops and Maintenance,30.084468,Closed,svcCRMSS,BANDERA RD and BRESNAHAN,7,2018
6,1014127338,2018-01-01 06:57:00,2018-01-02 15:32:00,2018-01-17 08:30:00,NO,-14.706736,YES,Code Enforcement,Front Or Side Yard Parking,16.064294,Closed,svcCRMSS,"10133 FIGARO CANYON, San Antonio, 78251",4,2018
7,1014127339,2018-01-01 06:58:00,2018-01-02 15:32:00,2018-01-17 08:30:00,NO,-14.706620,YES,Code Enforcement,Front Or Side Yard Parking,16.063796,Closed,svcCRMSS,"10133 FIGARO CANYON, San Antonio, 78251",4,2018
8,1014127340,2018-01-01 06:58:00,2018-01-02 15:32:00,2018-01-17 08:30:00,NO,-14.706620,YES,Code Enforcement,Right Of Way/Sidewalk Obstruction,16.063333,Closed,svcCRMSS,"10133 FIGARO CANYON, San Antonio, 78251",4,2018
9,1014127341,2018-01-01 06:59:00,2018-01-02 15:32:00,2018-01-17 08:30:00,NO,-14.706493,YES,Code Enforcement,Front Or Side Yard Parking,16.062859,Closed,svcCRMSS,"10133 FIGARO CANYON, San Antonio, 78251",4,2018


### Convert num_days_late from days to hours in new columns num_hours_late.


In [106]:
df_case = df_case.withColumn('num_hours_late', df_case.num_days_late * 24)

In [107]:
df_case.show()

+----------+-------------------+----------------+-------------------+---------+-------------------+-----------+----------------+--------------------+------------------+-----------+---------+--------------------+----------------+-------------------+----+
|   case_id|   case_opened_date|case_closed_date|       SLA_due_date|case_late|      num_days_late|case_closed|   dept_division|service_request_type|          SLA_days|case_status|source_id|     request_address|council_district|     num_hours_late|year|
+----------+-------------------+----------------+-------------------+---------+-------------------+-----------+----------------+--------------------+------------------+-----------+---------+--------------------+----------------+-------------------+----+
|1014127332|2018-01-01 00:42:00|            2018|2020-09-26 00:42:00|       NO| -998.5087616000001|        YES|Field Operations|        Stray Animal|             999.0|     Closed| svcCRMLS|2315  EL PASO ST,...|               5|     -2396

### Convert the case_late column to a boolean column.


In [108]:
df_case.withColumn('case_late', df_case.case_late == "YES" ).show()

+----------+-------------------+----------------+-------------------+---------+-------------------+-----------+----------------+--------------------+------------------+-----------+---------+--------------------+----------------+-------------------+----+
|   case_id|   case_opened_date|case_closed_date|       SLA_due_date|case_late|      num_days_late|case_closed|   dept_division|service_request_type|          SLA_days|case_status|source_id|     request_address|council_district|     num_hours_late|year|
+----------+-------------------+----------------+-------------------+---------+-------------------+-----------+----------------+--------------------+------------------+-----------+---------+--------------------+----------------+-------------------+----+
|1014127332|2018-01-01 00:42:00|            2018|2020-09-26 00:42:00|    false| -998.5087616000001|        YES|Field Operations|        Stray Animal|             999.0|     Closed| svcCRMLS|2315  EL PASO ST,...|               5|     -2396

In [109]:
df_case = df_case.withColumn('case_late', df_case.case_late == "YES" )

### Convert the SLA_days columns to a double column.


In [111]:
df_case = df_case.withColumn("SLA_days", df_case.SLA_days.cast("double"))
df_case.printSchema()


root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: timestamp (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- SLA_due_date: timestamp (nullable = true)
 |-- case_late: boolean (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: integer (nullable = true)
 |-- num_hours_late: double (nullable = true)
 |-- year: string (nullable = true)



### Pull it all together (again)

In [112]:
df_join2 = df_case.join(df_dept, "dept_division", "inner")
df_join2.show()

+----------------+----------+-------------------+----------------+-------------------+---------+-------------------+-----------+--------------------+------------------+-----------+---------+--------------------+----------------+-------------------+----+--------------------+----------------------+-------------------+
|   dept_division|   case_id|   case_opened_date|case_closed_date|       SLA_due_date|case_late|      num_days_late|case_closed|service_request_type|          SLA_days|case_status|source_id|     request_address|council_district|     num_hours_late|year|           dept_name|standardized_dept_name|dept_subject_to_SLA|
+----------------+----------+-------------------+----------------+-------------------+---------+-------------------+-----------+--------------------+------------------+-----------+---------+--------------------+----------------+-------------------+----+--------------------+----------------------+-------------------+
|Field Operations|1014127332|2018-01-01 00:42:

### Create a DataFrame with all combinations of council_district and service_request_type (regardless of whether the combination is observed in the data).


In [114]:
df_allcombos = df_case.join(df_dept, df_case.dept_division == df_dept.dept_division, "full_outer")
df_allcombos.show(3)


+----------+-------------------+----------------+-------------------+---------+-------------+-----------+-------------+--------------------+--------+-----------+---------+--------------------+----------------+-------------------+----+-------------+--------------------+----------------------+-------------------+
|   case_id|   case_opened_date|case_closed_date|       SLA_due_date|case_late|num_days_late|case_closed|dept_division|service_request_type|SLA_days|case_status|source_id|     request_address|council_district|     num_hours_late|year|dept_division|           dept_name|standardized_dept_name|dept_subject_to_SLA|
+----------+-------------------+----------------+-------------------+---------+-------------+-----------+-------------+--------------------+--------+-----------+---------+--------------------+----------------+-------------------+----+-------------+--------------------+----------------------+-------------------+
|1014127378|2018-01-01 08:24:00|            2018|2018-01-03 0

### Join the case data with the source and department data.


In [116]:
df_source = spark.read.csv('./sa311/source.csv', sep=",", header=True, inferSchema=True)

In [117]:
df_source.show(2)

+---------+----------------+
|source_id| source_username|
+---------+----------------+
|   100137|Merlene Blodgett|
|   103582|     Carmen Cura|
+---------+----------------+
only showing top 2 rows



In [115]:
df_join2.show()

+----------------+----------+-------------------+----------------+-------------------+---------+-------------------+-----------+--------------------+------------------+-----------+---------+--------------------+----------------+-------------------+----+--------------------+----------------------+-------------------+
|   dept_division|   case_id|   case_opened_date|case_closed_date|       SLA_due_date|case_late|      num_days_late|case_closed|service_request_type|          SLA_days|case_status|source_id|     request_address|council_district|     num_hours_late|year|           dept_name|standardized_dept_name|dept_subject_to_SLA|
+----------------+----------+-------------------+----------------+-------------------+---------+-------------------+-----------+--------------------+------------------+-----------+---------+--------------------+----------------+-------------------+----+--------------------+----------------------+-------------------+
|Field Operations|1014127332|2018-01-01 00:42:

In [122]:
df_all_files = df_join2.join(df_source, df_join2.source_id == df_source.source_id, 'left_outer')
df_all_files.show()

+----------------+----------+-------------------+----------------+-------------------+---------+-------------------+-----------+--------------------+------------------+-----------+---------+--------------------+----------------+-------------------+----+--------------------+----------------------+-------------------+---------+---------------+
|   dept_division|   case_id|   case_opened_date|case_closed_date|       SLA_due_date|case_late|      num_days_late|case_closed|service_request_type|          SLA_days|case_status|source_id|     request_address|council_district|     num_hours_late|year|           dept_name|standardized_dept_name|dept_subject_to_SLA|source_id|source_username|
+----------------+----------+-------------------+----------------+-------------------+---------+-------------------+-----------+--------------------+------------------+-----------+---------+--------------------+----------------+-------------------+----+--------------------+----------------------+---------------

### Are there any cases that do not have a request source?


In [123]:
df_join2.join(df_source, df_join2.source_id == df_source.source_id, "left_anti").show()

+-------------+-------+----------------+----------------+------------+---------+-------------+-----------+--------------------+--------+-----------+---------+---------------+----------------+--------------+----+---------+----------------------+-------------------+
|dept_division|case_id|case_opened_date|case_closed_date|SLA_due_date|case_late|num_days_late|case_closed|service_request_type|SLA_days|case_status|source_id|request_address|council_district|num_hours_late|year|dept_name|standardized_dept_name|dept_subject_to_SLA|
+-------------+-------+----------------+----------------+------------+---------+-------------+-----------+--------------------+--------+-----------+---------+---------------+----------------+--------------+----+---------+----------------------+-------------------+
+-------------+-------+----------------+----------------+------------+---------+-------------+-----------+--------------------+--------+-----------+---------+---------------+----------------+--------------

## No

### Who are the top 10 service request types in terms of number of requests?


In [126]:
df_all_files.toPandas().head()

,dept_division,case_id,case_opened_date,case_closed_date,SLA_due_date,case_late,num_days_late,case_closed,service_request_type,SLA_days,...,source_id,request_address,council_district,num_hours_late,year,dept_name,standardized_dept_name,dept_subject_to_SLA,source_id,source_username
0,Field Operations,1014127332,2018-01-01 00:42:00,2018,2020-09-26 00:42:00,False,-998.508762,YES,Stray Animal,999.000000,...,svcCRMLS,"2315 EL PASO ST, San Antonio, 78207",5,-23964.210278,2018,Animal Care Services,Animal Care Services,YES,svcCRMLS,svcCRMLS
1,Storm Water,1014127333,2018-01-01 00:46:00,2018,2018-01-05 08:30:00,False,-2.012604,YES,Removal Of Obstruction,4.322222,...,svcCRMSS,"2215 GOLIAD RD, San Antonio, 78223",3,-48.302500,2018,Trans & Cap Improvements,Trans & Cap Improvements,YES,svcCRMSS,svcCRMSS
2,Storm Water,1014127334,2018-01-01 00:48:00,2018,2018-01-05 08:30:00,False,-3.022338,YES,Removal Of Obstruction,4.320729,...,svcCRMSS,"102 PALFREY ST W, San Antonio, 78223",3,-72.536111,2018,Trans & Cap Improvements,Trans & Cap Improvements,YES,svcCRMSS,svcCRMSS
3,Code Enforcement,1014127335,2018-01-01 01:29:00,2018,2018-01-17 08:30:00,False,-15.011481,YES,Front Or Side Yard Parking,16.291887,...,svcCRMSS,"114 LA GARDE ST, San Antonio, 78223",3,-360.275556,2018,Code Enforcement Services,DSD/Code Enforcement,YES,svcCRMSS,svcCRMSS
4,Field Operations,1014127336,2018-01-01 01:34:00,2018,2018-01-01 04:34:00,True,0.372164,YES,Animal Cruelty(Critical),0.125000,...,svcCRMSS,"734 CLEARVIEW DR, San Antonio, 78228",7,8.931944,2018,Animal Care Services,Animal Care Services,YES,svcCRMSS,svcCRMSS


In [152]:
df_all_files.dtypes

[('dept_division', 'string'),
 ('case_id', 'int'),
 ('case_opened_date', 'timestamp'),
 ('case_closed_date', 'string'),
 ('SLA_due_date', 'timestamp'),
 ('case_late', 'boolean'),
 ('num_days_late', 'double'),
 ('case_closed', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'int'),
 ('num_hours_late', 'double'),
 ('year', 'string'),
 ('dept_name', 'string'),
 ('standardized_dept_name', 'string'),
 ('dept_subject_to_SLA', 'string'),
 ('source_id', 'string'),
 ('source_username', 'string')]

In [151]:
top_service_requests = df_all_files.groupBy('service_request_type').agg(count('service_request_type').alias('count')).sort('count',ascending=False)
top_service_requests.show(10)

+--------------------+-----+
|service_request_type|count|
+--------------------+-----+
|           No Pickup|89210|
|Overgrown Yard/Trash|66403|
|        Bandit Signs|32968|
|        Damaged Cart|31163|
|Front Or Side Yar...|28920|
|        Stray Animal|27361|
|Aggressive Animal...|25492|
|Cart Exchange Req...|22608|
|Junk Vehicle On P...|21649|
|     Pot Hole Repair|20827|
+--------------------+-----+
only showing top 10 rows



### Who are the top 10 service request types in terms of average days late?

In [156]:
top_service_late = df_all_files.groupBy('service_request_type').agg(mean('num_days_late').alias('avg_days_late')).sort('avg_days_late',ascending=False)
top_service_late.show(10)

+--------------------+------------------+
|service_request_type|     avg_days_late|
+--------------------+------------------+
|  Zoning: Junk Yards| 175.9563621042095|
|Labeling for Used...|162.43032902285717|
|Record Keeping of...|153.99724039428568|
|Signage Requied f...|151.63868055333333|
|Storage of Used M...|     142.112556415|
|Zoning: Recycle Yard|135.92851612479797|
|Donation Containe...|131.75610506358706|
|License Requied U...|128.79828704142858|
|Traffic Signal Gr...|101.79846062200002|
|           Complaint| 72.87050230311695|
+--------------------+------------------+
only showing top 10 rows



### Does number of days late depend on department?


In [157]:
top_service_late_bydept = df_all_files.groupBy('service_request_type', 'dept_division').agg(mean('num_days_late').alias('avg_days_late')).sort('avg_days_late',ascending=False)
top_service_late_bydept.show(10)

+--------------------+--------------------+------------------+
|service_request_type|       dept_division|     avg_days_late|
+--------------------+--------------------+------------------+
|  Zoning: Junk Yards|    Code Enforcement| 175.9563621042095|
|Labeling for Used...|Code Enforcement ...|162.43032902285717|
|Record Keeping of...|Code Enforcement ...|153.99724039428568|
|Signage Requied f...|Code Enforcement ...|151.63868055333333|
|Storage of Used M...|Code Enforcement ...|     142.112556415|
|Zoning: Recycle Yard|Code Enforcement ...|135.92851612479797|
|Donation Containe...|    Code Enforcement|131.75610506358706|
|License Requied U...|Code Enforcement ...|128.79828704142858|
|Traffic Signal Gr...|             Signals|101.79846062200002|
|Used Mattress Ins...|Code Enforcement ...| 92.93570056835293|
+--------------------+--------------------+------------------+
only showing top 10 rows



### It seems like Code Enforcement Department has a lot of very late request types...

In [159]:
top_dept_late = df_all_files.groupBy('dept_division').agg(mean('num_days_late').alias('avg_days_late')).sort('avg_days_late',ascending=False)
top_dept_late.show(10)

+--------------------+------------------+
|       dept_division|     avg_days_late|
+--------------------+------------------+
|Code Enforcement ...|135.92851612479797|
|        Reservations|       66.03116319|
|     311 Call Center|59.737091496300785|
|Director's Office...|37.570646702950086|
|Engineering Division| 13.43372455586971|
|               Shops|  9.64126176872269|
|           Tree Crew| 4.723282812065399|
|         Solid Waste|3.5190239198762248|
|              Trades| 3.231977141276932|
|Clean and Green N...| 1.691468919487805|
+--------------------+------------------+
only showing top 10 rows



### How do number of days late depend on department division and request type?

In [163]:
top_dept_late_byservice = df_all_files.groupBy('dept_division', 'service_request_type').agg(mean('num_days_late').alias('avg_days_late')).sort('avg_days_late',ascending=False)
top_dept_late_byservice.show(10)

+--------------------+--------------------+------------------+
|       dept_division|service_request_type|     avg_days_late|
+--------------------+--------------------+------------------+
|    Code Enforcement|  Zoning: Junk Yards| 175.9563621042095|
|Code Enforcement ...|Labeling for Used...|162.43032902285717|
|Code Enforcement ...|Record Keeping of...|153.99724039428568|
|Code Enforcement ...|Signage Requied f...|151.63868055333333|
|Code Enforcement ...|Storage of Used M...|     142.112556415|
|Code Enforcement ...|Zoning: Recycle Yard|135.92851612479797|
|    Code Enforcement|Donation Containe...|131.75610506358706|
|Code Enforcement ...|License Requied U...|128.79828704142858|
|             Signals|Traffic Signal Gr...|101.79846062200002|
|Code Enforcement ...|Used Mattress Ins...| 92.93570056835293|
+--------------------+--------------------+------------------+
only showing top 10 rows

